In [4]:
%load_ext autoreload
%autoreload 2
from innolux import *


In [5]:
condition_table = pl.read_excel(data_path / "T1 6634 6053 製程改善" / "conditions.xlsx" )
qtime_raw = pl.read_excel(data_path / "T1 6634 6053 製程改善" / "t1_qtime_raw.xlsx")


Could not determine dtype for column 3, falling back to string


In [6]:
condition_table

條件編號,條件,TFT,CF
i64,str,str,str
1,"""PAUV 224 mJ(出力 100%)""","""654A1AP80Q3002""","""654A1FP810W045"""
2,"""PAUV 224 mJ(出力 70%)""","""654A1AP80DH004""","""654A1FP810D069"""
3,"""PAUV 280 mJ(出力 100%)""","""654A1AP80PX020""","""654A1FP810A023"""
4,"""PAUV 280 mJ(出力 70%)(同量產條件)""","""654A1AP813V005""","""654A1FP810N010"""
6,"""PAUV 336 mJ(出力 70%)""","""654A1AP80Q3004""","""654A1FP810T030"""
…,…,…,…
9,"""ODF 加抽 85 sec""","""654A1AP80DL001""","""654A1FP810S033"""
10,"""SEUV 3600""","""654A1AP80DL004""","""654A1FP810R050"""
11,"""SEUV 5000""","""654A1AP80DL005""","""654A1FP8113006"""


In [16]:
from innolux.fabs import fabs
t1 = fabs['T1']["q-time"]

In [17]:
qtime_1 = qtime_raw.select(
    pl.col("COMPONENT_ID").alias("ID"),
).unique()
for step, stations in t1.items(): 
    qtime_1 = qtime_1.join(
        qtime_raw.filter(
            pl.col('SUB_EQUIP_ID').is_in(stations)
        )
        .sort("TRACK_OUT_TIME")
        .group_by("COMPONENT_ID", maintain_order=True).last()
        .select(
            pl.col("COMPONENT_ID").alias("ID"),
            pl.col("TRACK_IN_TIME").alias(f"{step}_IN"),
            pl.col("TRACK_OUT_TIME").alias(f"{step}_OUT"),
        ),
        on='ID',
        how="left",
    )

In [18]:
qtime_1

ID,PIPB_IN,PIPB_OUT,PAUV_IN,PAUV_OUT,PAPB_IN,PAPB_OUT,BOOX_IN,BOOX_OUT,LCDP_IN,LCDP_OUT,ODFX_IN,ODFX_OUT,SEUV_IN,SEUV_OUT
str,datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms]
"""654A1FP810A023""",2025-08-26 23:49:20,2025-08-27 00:02:32,2025-08-27 11:33:49,2025-08-27 11:33:56,2025-08-27 11:49:13,2025-08-27 12:19:17,null,null,null,null,null,null,null,null
"""654A1AP80Q0021""",2025-08-26 00:15:17,2025-08-26 00:25:36,2025-08-26 02:08:38,2025-08-26 02:08:45,2025-08-26 02:26:17,2025-08-26 02:56:33,2025-08-26 20:20:55,2025-08-26 20:21:02,2025-08-26 20:37:12,2025-08-26 20:38:59,2025-08-26 20:40:29,2025-08-26 20:44:05,2025-08-26 20:46:16,2025-08-26 20:47:50
"""654A1FP810N049""",2025-08-26 19:47:43,2025-08-26 19:58:01,2025-08-26 23:42:01,2025-08-26 23:42:09,2025-08-27 00:59:58,2025-08-27 01:30:19,null,null,null,null,null,null,null,null
"""654A1AP80PX020""",2025-08-26 17:12:18,2025-08-26 17:22:37,2025-08-27 12:09:12,2025-08-27 12:09:21,2025-08-27 12:19:44,2025-08-27 12:49:46,2025-08-27 18:19:40,2025-08-27 18:19:47,2025-08-27 18:34:51,2025-08-27 18:37:05,2025-08-27 18:39:27,2025-08-27 18:42:41,2025-08-27 18:45:25,2025-08-27 18:47:04
"""654A1AP80SX022""",2025-08-26 01:05:03,2025-08-26 01:18:39,2025-08-26 03:01:10,2025-08-26 03:01:19,2025-08-26 03:56:05,2025-08-26 04:26:07,2025-08-26 23:18:17,2025-08-26 23:18:24,2025-08-26 23:33:39,2025-08-26 23:35:20,2025-08-26 23:36:55,2025-08-26 23:40:42,2025-08-26 23:42:51,2025-08-26 23:44:27
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""654A1AP80Q3002""",2025-08-26 17:13:09,2025-08-26 17:23:26,2025-08-27 11:57:29,2025-08-27 11:57:38,2025-08-27 12:20:39,2025-08-27 12:50:39,2025-08-27 17:35:31,2025-08-27 17:35:38,2025-08-27 17:51:00,2025-08-27 17:53:15,2025-08-27 17:55:22,2025-08-27 17:58:32,2025-08-27 18:00:35,2025-08-27 18:02:14
"""654A1AP80DL006""",2025-08-26 13:29:00,2025-08-26 13:39:15,2025-08-26 19:28:31,2025-08-26 19:28:38,2025-08-26 19:45:45,2025-08-26 20:16:05,2025-08-27 16:24:17,2025-08-27 16:24:24,2025-08-27 16:46:51,2025-08-27 16:52:15,2025-08-27 16:55:15,2025-08-27 16:58:58,2025-08-27 17:01:06,2025-08-27 17:02:57
"""654A1AP80T0024""",2025-08-26 13:30:22,2025-08-26 13:40:55,2025-08-26 19:25:29,2025-08-26 19:25:36,2025-08-26 19:47:31,2025-08-26 20:17:35,2025-08-27 15:59:01,2025-08-27 15:59:08,2025-08-27 16:15:38,2025-08-27 16:17:30,2025-08-27 16:20:49,2025-08-27 16:24:34,2025-08-27 16:26:45,2025-08-27 16:28:19


In [12]:
(
    qtime_raw.filter(
        # pl.col('SUB_EQUIP_ID').is_in(stations),
        pl.col('SUB_EQUIP_ID').str.contains("1CPIL")
    )
    .sort("TRACK_OUT_TIME")
    .group_by("COMPONENT_ID", maintain_order=True).last()
    .select(
        "SUB_EQUIP_ID",
        pl.col("COMPONENT_ID").alias("ID"),
        pl.col("TRACK_IN_TIME").alias(f"{step}_IN"),
        pl.col("TRACK_OUT_TIME").alias(f"{step}_OUT"),
    )
)

SUB_EQUIP_ID,ID,PIPB_IN,PIPB_OUT
str,str,datetime[ms],datetime[ms]
"""1CPIL200.P4""","""654A1FP810B020""",2025-08-25 21:24:31,2025-08-25 21:24:48
"""1CPIL200.P4""","""654A1FP8109044""",2025-08-25 21:25:05,2025-08-25 21:25:23
"""1CPIL2B0""","""654A1FP810J029""",2025-08-25 21:30:51,2025-08-25 21:31:08
"""1CPIL2B0""","""654A1FP810J015""",2025-08-25 21:31:28,2025-08-25 21:31:47
"""1CPIL2B0""","""654A1FP810F055""",2025-08-25 21:31:38,2025-08-25 21:31:55
…,…,…,…
"""1CPIL200.P5""","""654A1FP810Q036""",2025-08-26 20:04:49,2025-08-26 20:05:07
"""1CPIL200""","""654A1FP810T030""",2025-08-26 23:38:54,2025-08-27 00:06:10
"""1CPIL200.P5""","""654A1FP810A023""",2025-08-27 00:06:44,2025-08-27 00:07:02


In [9]:
qtime_1

ID,PIPB_IN,PIPB_OUT,PAUV_IN,PAUV_OUT,PAPB_IN,PAPB_OUT,BOOX_IN,BOOX_OUT,LCDP_IN,LCDP_OUT,ODFX_IN,ODFX_OUT,SEUV_IN,SEUV_OUT
str,datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms]
"""654A1FP810N010""",2025-08-26 19:08:03,2025-08-26 19:18:13,2025-08-26 22:07:18,2025-08-26 22:07:25,2025-08-26 23:53:50,2025-08-27 00:24:13,null,null,null,null,null,null,null,null
"""654A1AP80Q3002""",2025-08-26 17:13:09,2025-08-26 17:23:26,2025-08-27 11:57:29,2025-08-27 11:57:38,2025-08-27 12:20:39,2025-08-27 12:50:39,2025-08-27 17:35:31,2025-08-27 17:35:38,2025-08-27 17:51:00,2025-08-27 17:53:15,2025-08-27 17:55:22,2025-08-27 17:58:32,2025-08-27 18:00:35,2025-08-27 18:02:14
"""654A1FP810J015""",2025-08-25 21:16:46,2025-08-25 21:27:15,2025-08-25 22:30:57,2025-08-25 22:31:07,2025-08-26 00:09:23,2025-08-26 00:39:31,null,null,null,null,null,null,null,null
"""654A1FP810S033""",2025-08-26 19:49:23,2025-08-26 19:59:29,2025-08-26 23:38:59,2025-08-26 23:39:07,2025-08-27 01:01:42,2025-08-27 01:31:53,null,null,null,null,null,null,null,null
"""654A1AP80DL006""",2025-08-26 13:29:00,2025-08-26 13:39:15,2025-08-26 19:28:31,2025-08-26 19:28:38,2025-08-26 19:45:45,2025-08-26 20:16:05,2025-08-27 16:24:17,2025-08-27 16:24:24,2025-08-27 16:46:51,2025-08-27 16:52:15,2025-08-27 16:55:15,2025-08-27 16:58:58,2025-08-27 17:01:06,2025-08-27 17:02:57
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""654A1AP80Q0019""",2025-08-26 00:19:35,2025-08-26 00:29:54,2025-08-26 02:07:57,2025-08-26 02:08:04,2025-08-26 02:26:53,2025-08-26 02:56:55,2025-08-26 20:19:09,2025-08-26 20:19:16,2025-08-26 20:34:39,2025-08-26 20:37:07,2025-08-26 20:38:45,2025-08-26 20:41:57,2025-08-26 20:44:00,2025-08-26 20:45:39
"""654A1FP810R050""",2025-08-26 19:48:35,2025-08-26 19:58:45,2025-08-26 23:39:44,2025-08-26 23:39:52,2025-08-27 01:01:42,2025-08-27 01:31:53,null,null,null,null,null,null,null,null
"""654A1AP80DL007""",2025-08-26 13:28:59,2025-08-26 13:39:14,2025-08-26 19:29:16,2025-08-26 19:29:23,2025-08-26 19:45:45,2025-08-26 20:16:05,2025-08-27 16:25:36,2025-08-27 16:25:43,2025-08-27 16:55:36,2025-08-27 16:57:56,2025-08-27 16:59:58,2025-08-27 17:03:32,2025-08-27 17:05:41,2025-08-27 17:07:41


In [31]:
qtime_2 = condition_table.join(
    qtime_1.select([col for col in qtime_1.columns if qtime_1[col].null_count() == 0]),
    left_on="CF",
    right_on='ID',
    suffix="_CF",
    maintain_order="left",
).join(
    qtime_1,
    left_on="TFT",
    right_on="ID",
    suffix="_TFT",
    maintain_order="left",
)

In [32]:
qtime_2

條件編號,條件,TFT,CF,PIPB_IN,PIPB_OUT,PAUV_IN,PAUV_OUT,PAPB_IN,PAPB_OUT,PIPB_IN_TFT,PIPB_OUT_TFT,PAUV_IN_TFT,PAUV_OUT_TFT,PAPB_IN_TFT,PAPB_OUT_TFT,BOOX_IN,BOOX_OUT,LCDP_IN,LCDP_OUT,ODFX_IN,ODFX_OUT,SEUV_IN,SEUV_OUT
i64,str,str,str,datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms],datetime[ms]
1,"""PAUV 224 mJ(出力 100%)""","""654A1AP80Q3002""","""654A1FP810W045""",2025-08-26 23:50:10,2025-08-27 00:03:10,2025-08-27 11:16:27,2025-08-27 11:16:35,2025-08-27 11:49:51,2025-08-27 12:20:13,2025-08-26 17:13:09,2025-08-26 17:23:26,2025-08-27 11:57:29,2025-08-27 11:57:38,2025-08-27 12:20:39,2025-08-27 12:50:39,2025-08-27 17:35:31,2025-08-27 17:35:38,2025-08-27 17:51:00,2025-08-27 17:53:15,2025-08-27 17:55:22,2025-08-27 17:58:32,2025-08-27 18:00:35,2025-08-27 18:02:14
2,"""PAUV 224 mJ(出力 70%)""","""654A1AP80DH004""","""654A1FP810D069""",2025-08-26 23:49:21,2025-08-27 00:02:33,2025-08-27 11:23:42,2025-08-27 11:23:49,2025-08-27 11:49:51,2025-08-27 12:20:13,2025-08-26 17:12:19,2025-08-26 17:22:38,2025-08-27 12:03:11,2025-08-27 12:03:20,2025-08-27 12:20:39,2025-08-27 12:50:39,2025-08-27 17:57:50,2025-08-27 17:57:56,2025-08-27 18:12:53,2025-08-27 18:15:07,2025-08-27 18:16:39,2025-08-27 18:20:26,2025-08-27 18:23:01,2025-08-27 18:24:41
3,"""PAUV 280 mJ(出力 100%)""","""654A1AP80PX020""","""654A1FP810A023""",2025-08-26 23:49:20,2025-08-27 00:02:32,2025-08-27 11:33:49,2025-08-27 11:33:56,2025-08-27 11:49:13,2025-08-27 12:19:17,2025-08-26 17:12:18,2025-08-26 17:22:37,2025-08-27 12:09:12,2025-08-27 12:09:21,2025-08-27 12:19:44,2025-08-27 12:49:46,2025-08-27 18:19:40,2025-08-27 18:19:47,2025-08-27 18:34:51,2025-08-27 18:37:05,2025-08-27 18:39:27,2025-08-27 18:42:41,2025-08-27 18:45:25,2025-08-27 18:47:04
4,"""PAUV 280 mJ(出力 70%)(同量產條件)""","""654A1AP813V005""","""654A1FP810N010""",2025-08-26 19:08:03,2025-08-26 19:18:13,2025-08-26 22:07:18,2025-08-26 22:07:25,2025-08-26 23:53:50,2025-08-27 00:24:13,2025-08-26 12:37:59,2025-08-26 12:48:23,2025-08-26 17:54:46,2025-08-26 17:54:54,2025-08-26 18:13:21,2025-08-26 18:43:31,2025-08-27 11:32:44,2025-08-27 11:44:33,2025-08-27 12:02:32,2025-08-27 12:04:54,2025-08-27 12:06:30,2025-08-27 12:10:20,2025-08-27 12:12:34,2025-08-27 12:14:15
6,"""PAUV 336 mJ(出力 70%)""","""654A1AP80Q3004""","""654A1FP810T030""",2025-08-26 23:48:33,2025-08-26 23:59:02,2025-08-27 11:40:43,2025-08-27 11:40:51,2025-08-27 11:49:13,2025-08-27 12:19:17,2025-08-26 17:11:28,2025-08-26 17:21:47,2025-08-27 12:15:37,2025-08-27 12:15:46,2025-08-27 12:19:44,2025-08-27 12:49:46,2025-08-27 18:39:45,2025-08-27 18:39:52,2025-08-27 18:54:52,2025-08-27 18:56:34,2025-08-27 18:59:15,2025-08-27 19:03:04,2025-08-27 19:05:16,2025-08-27 19:06:54
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
9,"""ODF 加抽 85 sec""","""654A1AP80DL001""","""654A1FP810S033""",2025-08-26 19:49:23,2025-08-26 19:59:29,2025-08-26 23:38:59,2025-08-26 23:39:07,2025-08-27 01:01:42,2025-08-27 01:31:53,2025-08-26 13:30:21,2025-08-26 13:40:54,2025-08-26 19:26:14,2025-08-26 19:26:21,2025-08-26 19:47:31,2025-08-26 20:17:35,2025-08-27 16:20:06,2025-08-27 16:20:13,2025-08-27 16:39:35,2025-08-27 16:41:21,2025-08-27 16:42:56,2025-08-27 16:47:13,2025-08-27 16:49:31,2025-08-27 16:51:12
10,"""SEUV 3600""","""654A1AP80DL004""","""654A1FP810R050""",2025-08-26 19:48:35,2025-08-26 19:58:45,2025-08-26 23:39:44,2025-08-26 23:39:52,2025-08-27 01:01:42,2025-08-27 01:31:53,2025-08-26 13:29:42,2025-08-26 13:40:07,2025-08-26 19:27:00,2025-08-26 19:27:07,2025-08-26 19:46:37,2025-08-26 20:16:58,2025-08-27 16:21:25,2025-08-27 16:21:32,2025-08-27 16:42:16,2025-08-27 16:44:29,2025-08-27 16:47:03,2025-08-27 16:51:53,2025-08-27 16:54:10,2025-08-27 16:55:27
11,"""SEUV 5000""","""654A1AP80DL005""","""654A1FP8113006""",2025-08-26 19:48:34,2025-08-26 19:58:44,2025-08-26 23:40:30,2025-08-26 23:40:38,2025-08-27 01:00:50,2025-08-27 01:30:53,20

In [33]:
q_time_3 = qtime_2.select(
    "條件", "TFT", "CF",
    (pl.col('PAUV_IN'    ) - pl.col('PIPB_OUT'    )).alias('PIPB to PAUV - C'),
    (pl.col('PAUV_IN_TFT') - pl.col('PIPB_OUT_TFT')).alias('PIPB to PAUV - T'),
    (pl.col('BOOX_IN'    ) - pl.col('PAPB_OUT'    )).alias('PAPB to BOOX - C'),
    (pl.col('BOOX_IN'    ) - pl.col('PAPB_OUT_TFT')).alias('PAPB to BOOX - T'),
    (pl.col('ODFX_IN'    ) - pl.col('LCDP_OUT'    )).alias('LCDP to ODFX'    ),
    (pl.col('SEUV_IN'    ) - pl.col('ODFX_OUT'    )).alias('ODFX to SEUV'    ),
)

In [34]:
q_time_3.write_excel("./q_time.xlsx")

In [29]:
q_time_3

條件,TFT,CF,PIPB to PAUV - C,PIPB to PAUV - T,PAPB to BOOX - C,PAPB to BOOX - T,LCDP to ODFX,ODFX to SEUV
str,str,str,duration[ms],duration[ms],duration[ms],duration[ms],duration[ms],duration[ms]
"""PAUV 224 mJ(出力 100%)""","""654A1AP80Q3002""","""654A1FP810W045""",11h 13m 17s,18h 34m 3s,5h 15m 18s,4h 44m 52s,2m 7s,2m 3s
"""SEUV 7000""","""654A1AP80DL006""","""654A1FP810C039""",3h 43m 13s,5h 49m 16s,14h 53m 24s,20h 8m 12s,3m,2m 8s
"""PAUV 336 mJ(出力 70%)""","""654A1AP80Q3004""","""654A1FP810T030""",11h 41m 41s,18h 53m 50s,6h 20m 28s,5h 49m 59s,2m 41s,2m 12s
"""LC Pattern 1""","""654A1AP80T0023""","""654A1FP810Q036""",3h 36m 55s,5h 42m 55s,14h 24m 49s,19h 39m 12s,1m 36s,2m 9s
"""SEUV 5000""","""654A1AP80DL005""","""654A1FP8113006""",3h 41m 46s,5h 47m 39s,14h 51m 51s,20h 5m 46s,4m 56s,2m 17s
…,…,…,…,…,…,…,…,…
"""SEUV 3600""","""654A1AP80DL004""","""654A1FP810R050""",3h 40m 59s,5h 46m 53s,14h 49m 32s,20h 4m 27s,2m 34s,2m 17s
"""PAUV 224 mJ(出力 70%)""","""654A1AP80DH004""","""654A1FP810D069""",11h 21m 9s,18h 40m 33s,5h 37m 37s,5h 7m 11s,1m 32s,2m 35s
"""PAUV 280 mJ(出力 100%)""","""654A1AP80PX020""","""654A1FP810A023""",11h 31m 17s,18h 46m 35s,6h 23s,5h 29m 54s,2m 22s,2m 44s


In [20]:
edc_raw = pl.read_excel(data_path / "T1 6634 6053 製程改善" / "t1_eda_raw.xlsx")
edc_raw

Could not determine dtype for column 7, falling back to string


STEP_ID,PARAM_COLLECTION,PARAM_NAME,COMPONENT_ID,TRACK_OUT_TIME,PRODUCT_ID,SUB_EQUIP_ID,CHAMBER_ID,BATCH_ID,GLASS_ID,PARAM_TYPE,RECIPE_NAME,OPERATOR_ID,PLAN_ID,EQUIP_ID,CARRIER_ID,FAB_AREA,PRODUCT_VERSION,AVG_VALUE,STD_VALUE,MAX_VALUE,MIN_VALUE,COUNT_VALUE,PARAM_VALUE,RN
i64,str,str,str,datetime[ms],str,str,str,str,str,i64,str,str,str,str,str,str,str,f64,f64,f64,f64,i64,str,i64
6300,"""PDSG_1CASM100_1CASM1H0""","""AC2_P_E1-3""","""654A1AP80DH004""",2025-08-27 20:41:36,"""654A1C""","""1CASM1H0""","""N/A""","""EF58270V""","""654A1AP80DH004""",2,"""R1_654F_T_01""","""InlineBC""","""654A1C_FB_A_T1_1""","""1CASM100""","""1D124""","""T1""",""" """,-2.0,0.0,-2.0,-2.0,1,""" """,1
6300,"""PDSG_1CASM100_1CASM1H0""","""AC2_P_E1-3""","""654A1AP80DL001""",2025-08-27 18:26:07,"""654A1C""","""1CASM1H0""","""N/A""","""EF58250Z""","""654A1AP80DL001""",2,"""R1_654F_T_01""","""InlineBC""","""654A1C_FB_A_T1_1""","""1CASM100""","""1D087""","""T1""",""" """,-21.0,0.0,-21.0,-21.0,1,""" """,2
6300,"""PDSG_1CASM100_1CASM1H0""","""AC2_P_E1-3""","""654A1AP80DL004""",2025-08-27 18:40:54,"""654A1C""","""1CASM1H0""","""N/A""","""EF58250Z""","""654A1AP80DL004""",2,"""R1_654F_T_01""","""InlineBC""","""654A1C_FB_A_T1_1""","""1CASM100""","""1D037""","""T1""",""" """,-14.0,0.0,-14.0,-14.0,1,""" """,3
6300,"""PDSG_1CASM100_1CASM1H0""","""AC2_P_E2-4""","""654A1AP80DH004""",2025-08-27 20:41:36,"""654A1C""","""1CASM1H0""","""N/A""","""EF58270V""","""654A1AP80DH004""",2,"""R1_654F_T_01""","""InlineBC""","""654A1C_FB_A_T1_1""","""1CASM100""","""1D124""","""T1""",""" """,12.0,0.0,12.0,12.0,1,""" """,4
6300,"""PDSG_1CASM100_1CASM1H0""","""AC2_P_E2-4""","""654A1AP80DL001""",2025-08-27 18:26:07,"""654A1C""","""1CASM1H0""","""N/A""","""EF58250Z""","""654A1AP80DL001""",2,"""R1_654F_T_01""","""InlineBC""","""654A1C_FB_A_T1_1""","""1CASM100""","""1D087""","""T1""",""" """,8.0,0.0,8.0,8.0,1,""" """,5
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
6100,"""PDSG_1CPIL200_1CPIL2A0""","""over_curing""","""654A1FP810Z070""",2025-08-26 20:10:36,"""654A1F00B_A""","""1CPIL2A0""","""N/A""","""EF58250Z""","""654A1FP810Z070""",2,"""R1_654F_F_01""","""InlineBC""","""654A1F00B_A_T1_1""","""1CPIL200""","""1C503""","""T1""",""" """,0.0,0.0,0.0,0.0,1,""" """,33896
6100,"""PDSG_1CPIL200_1CPIL2A0""","""over_curing""","""654A1FP8113006""",2025-08-26 20:10:36,"""654A1F00B_A""","""1CPIL2A0""","""N/A""","""EF58250Z""","""654A1FP8113006""",2,"""R1_654F_F_01""","""InlineBC""","""654A1F00B_A_T1_1""","""1CPIL200""","""1C503""","""T1""",""" """,0.0,0.0,0.0,0.0,1,""" """,33897
6100,"""PDSG_1CPIL200_1CPIL2A0""","""over_curing""","""654A1AP80PX020""",2025-08-26 17:34:49,"""654A1C""","""1CPIL2A0""","""N/A""","""EF58250Z""","""654A1AP80PX020""",2,"""R1_654F_T_01""","""InlineBC""","""654A1C_FB_A_T1_1""","""1CPIL200""","""1C119""","""T1""",""" """,0.0,0.0,0.0,0.0,1,""" """,33898


In [21]:
edc_1 = edc_raw.select(
    "GLASS_ID",
    "STEP_ID",
    "SUB_EQUIP_ID",
    "TRACK_OUT_TIME",
    "PARAM_NAME",
    "AVG_VALUE",
)

In [22]:
# edc_steps = {
#     "PIPR TEMP": {"step": ("1CPIL", "PreCur[1-4]_temp"), "filter": None},
#     "PIPR TIME": {"step": ("1CPIL", "Heater_Time"),      "filter": None},
#     "PIPB TEMP": {"step": ("1CPIL", "Oven[A-D]_Temp"),   "filter": None},
#     "PIPB TIME": {"step": ("1CPIL", "Heating_Time"),     "filter": None},
#     "PAUV"     : {"step": ("1CPHA", "S[1-2]_EXP_MON"),   "filter": [pl.col("AVG_VALUE") > 100]},
#     "PAPB TEMP": {"step": ("1CPAO", "Heating_Temp"),     "filter": None},
#     "PAPB TIME": {"step": ("1CPAO", "Heating_Time"),     "filter": None},
#     "BOOX TEMP": {"step": ("1CASM", "Current_Temp"),     "filter": None},
#     "BOOX TIME": {"step": ("1CASM", "HEAT_TIME"),        "filter": None},
#     "ODFX Q2"  : {"step": ("1CASM", "ASMQ2_TIME"),       "filter": None}, # ODFX in -> SEUV out
#     "SEUV"     : {"step": ("1CASM", "(?i)uv_energy_2"),  "filter": None},
#     "SEPB TEMP": {"step": ("1CASM", "Oven_temp"),        "filter": None},
#     "SEPB TIME": {"step": ("1CASM", "HeatingTime"),      "filter": None},
# }

def get_val(df: pl.DataFrame, step: tuple[str, str], filter: list[pl.Expr] | None = None) -> pl.DataFrame:
    result = (
        df.filter(
            pl.col("SUB_EQUIP_ID").str.contains(step[0]),
            pl.col("PARAM_NAME").str.contains(step[1])
        )
        # .select(
        #     "GLASS_ID", "SUB_EQUIP_ID", "PARAM_NAME", "AVG_VALUE"
        # )
        .select(
            "GLASS_ID", "AVG_VALUE"
        )
    )
    
    if filter is not None:
        result = result.filter(*filter)
    
    return result.group_by(['GLASS_ID']).mean()

In [23]:
edc_2 = edc_1.select(
    pl.col("GLASS_ID").alias("ID"),
).unique()

for step, kwargs in fabs["T1"]["edc"].items():
    edc_2 = edc_2.join(
        get_val(edc_1, **kwargs)
        .select(
            pl.col("GLASS_ID").alias("ID"),
            pl.col("AVG_VALUE").alias(step),
        ),
        on='ID',
        how='left'
    )
edc_2

ID,PIPR TEMP,PIPR TIME,PIPB TEMP,PIPB TIME,PAUV,PAPB TEMP,PAPB TIME,BOOX TEMP,BOOX TIME,ODFX Q2,SEUV,SEPB TEMP,SEPB TIME
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""654A1AP80DL004""",135.0,70.0,211.25,624.0,278.0,229.8,1821.0,150.125,626.0,504.0,3600.0,119.0,3579.0
"""654A1FP810R050""",135.0,70.0,211.333333,609.0,280.0,229.9,1812.0,null,null,null,null,null,null
"""654A1FP810A023""",135.0,70.0,211.333333,790.0,281.666667,231.0,1804.0,null,null,null,null,null,null
"""654A1AP80DH004""",135.0,70.0,211.166667,619.0,221.666667,230.4,1801.0,150.083333,625.0,482.0,6010.0,119.0,3578.0
"""654A1FP810W045""",135.0,70.0,211.333333,780.0,225.666667,230.5,1822.0,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""654A1FP810Z070""",135.0,70.0,211.333333,606.0,280.0,230.0,1820.0,null,null,null,null,null,null
"""654A1FP810D069""",135.0,70.0,211.333333,790.0,222.333333,231.0,1822.0,null,null,null,null,null,null
"""654A1AP80T0023""",135.0,70.0,211.25,641.0,280.0,230.3,1809.0,150.116667,625.0,454.0,6000.0,120.0,3578.0


In [26]:
edc_3 = condition_table.join(
    edc_2.select([col for col in edc_2.columns if edc_2[col].null_count() == 0]),
    left_on="CF",
    right_on='ID',
    suffix="_CF",
    maintain_order="left",
).join(
    edc_2,
    left_on="TFT",
    right_on="ID",
    suffix=" - T",
    maintain_order="left",
)

In [27]:
edc_4 = edc_3.select(
    pl.col('條件編號'),
    pl.col('條件'),
    pl.col('TFT'),
    pl.col('CF'),
    pl.col('PIPR TEMP - T'),
    pl.col('PIPR TEMP').alias('PIPR TEMP - C'),
    pl.col('PIPR TIME - T'),
    pl.col('PIPR TIME').alias('PIPR TIME - C'),
    pl.col('PIPB TEMP - T'),
    pl.col('PIPB TEMP').alias('PIPB TEMP - C'),
    pl.col('PIPB TIME - T'),
    pl.col('PIPB TIME').alias('PIPB TIME - C'),
    pl.col('PAUV - T'),
    pl.col('PAUV').alias('PAUV - C'),
    pl.col('PAPB TEMP - T'),
    pl.col('PAPB TEMP').alias('PAPB TEMP - C'),
    pl.col('PAPB TIME - T'),
    pl.col('PAPB TIME').alias('PAPB TIME - C'),
    pl.col('BOOX TEMP'),
    pl.col('BOOX TIME'),
    pl.col('ODFX Q2'),
    pl.col('SEUV'),
    pl.col('SEPB TEMP'),
    pl.col('SEPB TIME'),
)

In [28]:
edc_4

條件編號,條件,TFT,CF,PIPR TEMP - T,PIPR TEMP - C,PIPR TIME - T,PIPR TIME - C,PIPB TEMP - T,PIPB TEMP - C,PIPB TIME - T,PIPB TIME - C,PAUV - T,PAUV - C,PAPB TEMP - T,PAPB TEMP - C,PAPB TIME - T,PAPB TIME - C,BOOX TEMP,BOOX TIME,ODFX Q2,SEUV,SEPB TEMP,SEPB TIME
i64,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,"""PAUV 224 mJ(出力 100%)""","""654A1AP80Q3002""","""654A1FP810W045""",135.0,135.0,70.0,70.0,211.166667,211.333333,619.0,780.0,225.333333,225.666667,229.9,230.5,1801.0,1822.0,150.083333,625.0,412.0,6010.0,120.0,3579.0
2,"""PAUV 224 mJ(出力 70%)""","""654A1AP80DH004""","""654A1FP810D069""",135.0,135.0,70.0,70.0,211.166667,211.333333,619.0,790.0,221.666667,222.333333,230.4,231.0,1801.0,1822.0,150.083333,625.0,482.0,6010.0,119.0,3578.0
3,"""PAUV 280 mJ(出力 100%)""","""654A1AP80PX020""","""654A1FP810A023""",135.0,135.0,70.0,70.0,211.166667,211.333333,619.0,790.0,281.0,281.666667,231.1,231.0,1802.0,1804.0,150.091667,625.0,457.0,6010.0,120.0,3619.0
4,"""PAUV 280 mJ(出力 70%)(同量產條件)""","""654A1AP813V005""","""654A1FP810N010""",135.0,135.0,70.0,70.0,211.0,211.166667,623.0,610.0,277.666667,279.666667,232.6,231.3,1811.0,1823.0,150.05,637.5,465.0,6000.0,118.0,3630.0
6,"""PAUV 336 mJ(出力 70%)""","""654A1AP80Q3004""","""654A1FP810T030""",135.0,135.0,70.0,70.0,211.166667,211.333333,619.0,629.0,342.333333,342.666667,230.4,230.1,1802.0,1804.0,150.075,625.0,459.0,6010.0,120.0,3592.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
9,"""ODF 加抽 85 sec""","""654A1AP80DL001""","""654A1FP810S033""",135.0,135.0,70.0,70.0,211.25,211.333333,633.0,606.0,279.666667,278.0,231.4,229.8,1805.0,1812.0,150.108333,625.0,496.0,6010.0,120.0,3579.0
10,"""SEUV 3600""","""654A1AP80DL004""","""654A1FP810R050""",135.0,135.0,70.0,70.0,211.25,211.333333,624.0,609.0,278.0,280.0,229.8,229.9,1821.0,1812.0,150.125,626.0,504.0,3600.0,119.0,3579.0
11,"""SEUV 5000""","""654A1AP80DL005""","""654A1FP8113006""",135.0,135.0,70.0,70.0,211.25,211.25,624.0,609.0,280.0,278.333333,230.9,230.7,1821.0,1803.0,150.133333,626.5,452.0,5010.0,120.0,3579.0


In [208]:
edc_4.write_excel('edc.xlsx')